<a href="https://colab.research.google.com/github/MdShamsuddinRaju/10min-School/blob/main/rag_bangla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages
!pip install pymupdf sentence-transformers faiss-cpu

In [ ]:
# Upload the Bangla PDF book (HSC26 Bangla 1st Paper)
from google.colab import files
uploaded = files.upload()

Saving HSC26-Bangla1st-Paper.pdf to HSC26-Bangla1st-Paper (1).pdf


In [ ]:
# Extract text using PyMuPDF (best for Bangla)
import fitz

pdf_path = next(iter(uploaded))  # Get uploaded filename
doc = fitz.open(pdf_path)
text = ''
for page in doc:
    text += page.get_text()

print(text[:1000])  # Preview extracted content

অনলাইন ব্যাচ সম্পর্কিত যেককাকনা জিজ্ঞাাসা ,
অপরিরিতা
আল ািয রিষয়
িাাং া
১ম পত্র
১। অনুপলেি িািা কী কলি জীরিকা রনিবাহ কিলতন?
ক) ডাক্তার্ি
খ) ওকালর্ত
গ) মাস্টার্ি
ঘ) ব্যব্সা
২। োোলক ভাগ্য দেিতাি প্রধান এলজন্ট ি াি কািণ, তাি-
ক) প্রর্তপজি
খ) প্রভাব্
 
গ) র্ব্চক্ষণতা
ঘ) কূট ব্ুর্ি
র্নকচি অনুকেদটি পক়ে ৩ ও ৪ সংখযক প্রকেি উিি দাও।
র্পতৃহীন দীপুি চাচাই র্িকলন পর্িব্াকিি কতিা। দীপু র্িজক্ষত হকলও তাি র্সিান্ত যনও াি ক্ষমতা র্িল না। চাচা 
তাি র্ব্ক ি উকদযাগ র্নকলও যেৌতুক র্নক  ব্া়োব্ার়্ে কিাি কািকণ কনযাি র্পতা অপমার্নত যব্াধ ককি র্ব্ক ি 
আকলাচনা যভকে যদন। দীপু যমক টিি ির্ব্ যদকখ মুগ্ধ হকলও তাি চাচাকক র্কিুই ব্লকত পাকিনর্ন।
৩। েীপুি িািাি সলে ‘অপরিরিতা' গ্লেি দকান িরিলেি রে  আলে?
ক) হর্িকিি
খ) মামাি
গ) র্িক্ষককি
ঘ) র্ব্নুি
৪। উক্ত িরিলে প্রাধানয দপলয়লে -
i) যদৌিাত্ম
ii) হীনম্মনযতা 
 
 
iii) যলাভ
র্নকচি যকানটি ঠিক?
ক। i ও ii 
 
খ। ii ও iii 
 
গ। i ও iii 
 
ঘ। i, ii ও iii
৫. অনুপলেি িয়স কত িেি?
ক) পঁর্চি 
 
খ) িাব্বিি 
 
গ) সাতাি 
 
ঘ) আটাি
প্রাক-মূলযা ন
কতগুকলা প্রকেি সঠিক উিি র্দকত পািকল?
SL

In [ ]:
# Clean and chunk the text
import re
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

text = re.sub(r'\n+', '\n', text)
text = re.sub(r' +', ' ', text)

# Chunk by paragraph (or split every 500 characters with overlap)
def chunk_text(text, chunk_size=500, overlap=50):
    chunks = []
    for i in range(0, len(text), chunk_size - overlap):
        chunks.append(text[i:i + chunk_size])
    return chunks

chunks = chunk_text(text)
print(f"Total Chunks: {len(chunks)}")
print(chunks[0])

Total Chunks: 182
অনলাইন ব্যাচ সম্পর্কিত যেককাকনা জিজ্ঞাাসা ,
অপরিরিতা
আল ািয রিষয়
িাাং া
১ম পত্র
১। অনুপলেি িািা কী কলি জীরিকা রনিবাহ কিলতন?
ক) ডাক্তার্ি
খ) ওকালর্ত
গ) মাস্টার্ি
ঘ) ব্যব্সা
২। োোলক ভাগ্য দেিতাি প্রধান এলজন্ট ি াি কািণ, তাি-
ক) প্রর্তপজি
খ) প্রভাব্
 
গ) র্ব্চক্ষণতা
ঘ) কূট ব্ুর্ি
র্নকচি অনুকেদটি পক়ে ৩ ও ৪ সংখযক প্রকেি উিি দাও।
র্পতৃহীন দীপুি চাচাই র্িকলন পর্িব্াকিি কতিা। দীপু র্িজক্ষত হকলও তাি র্সিান্ত যনও াি ক্ষমতা র্িল না। চাচা 
তাি র্ব্ক ি উকদযাগ র্নকলও যেৌতুক র্নক ব্া়োব্ার়্ে কিাি কািকণ কন


In [ ]:
# Generate embeddings and create FAISS index
import faiss

model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')  # Bangla + English support
embeddings = model.encode(chunks)

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

In [ ]:
# Ask a question in Bangla or English
def ask_question(query, top_k=1):
    query_vec = model.encode([query])
    D, I = index.search(np.array(query_vec), top_k)
    return chunks[I[0][0]]

# Sample Questions
questions = [
    "অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?",
    "কাকে অনুপমের ভাগ্য দেবতা বলে উল্লেখ করা হয়েছে?",
    "বিয়ের সময় কল্যাণীর প্রকৃত বয়স কত ছিল?"
]

for q in questions:
    print(f"Q: {q}\nA (from context): {ask_question(q)}\n")

Q: অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?
A (from context):  যদাহািিা ব্ািব্াি পর্িকব্িন ককি।
ির়্েমা
আ়েষ্ট্তা। ি়েত্ব।
মঞ্জিী
র্কিল েুক্ত কর্চ ডাল। মুকুল
একপিন
একপ্রি
কানপুি
কলযাণী যে যদিমাতৃকাি যসব্া আত্মর্নক াগ ককিকি,
অনুপকমি এই আকত্মাপলর্ি এখাকন প্রকার্িত।
5
েূ গ্ে
আিআমািব্ সসাতািমাত্র।এিীব্নটানাদদকঘিযির্হসাকব্ব়্ে, নাগুকনির্হসাকব্।তব্ুইহািএকটুর্ব্কিষ
মূলযআকি।ইহাযসইফুকলিমকতাোহািব্ুককিউপকিভ্রমিআর্স াব্র্স ার্িল, এব্ংযসইপদকক্ষকপিইর্তহাস
তাহািিীব্কনিমাঝখাকনফকলিমকতাগুটিধর্ি াউঠি াকি।
যসইইর্তহাসটুকুআকাকিযিাকটা, তাহাককযিাকটাকর্ি াইজলর্খব্।যিাকটাককেঁাহািা

Q: কাকে অনুপমের ভাগ্য দেবতা বলে উল্লেখ করা হয়েছে?
A (from context):  তাহকল তাি অসম্পূণি সািব্িা প্রকট হক 
ওকঠ। র্নকিি র্নষ্ফল িীব্নকক যব্াঝাকত অনুপকমি ব্যব্হৃত
উপমা।
অন্নপূণিা
অকন্ন পর্িপূণিা। যদব্ী দুগিা।
গিানন
যদব্ী দুগিাি দুই পুত্র; অগ্রি গকণি ও অনুি কার্তিকক ।
দুগিাি যকাকল থাকা যদব্-যসনাপর্ত কার্তিকক ককযব্াঝাকনা
হক কি। ব্যঙ্গাকথি প্রক াগ।
আিও আমাকক যদর্খকল মকন হইকব্, 
আর্ম অন্নপূণিাি যকাকল গিানকনি 
যিাকটা ভাইটি।
ভূষণ, প্রসাধন, য